In [ ]:
%pip install pandas

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
top_n = 200
# Replace 'filename.csv' with your file path
df = pd.read_csv('~/Downloads/ps/reviews.csv')
df = df.dropna()

df = df.head(top_n)

print(len(df))
df

In [ ]:
%pip install openai tenacity python-dotenv

from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_random_exponential
from typing import List
import openai
import os

load_dotenv()

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, model, **kwargs) -> List[float]:

    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")
    text = text.replace("<br/>", " ")

    response = openai.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

In [ ]:
text="great location!"
model="text-embedding-ada-002"
embedding=get_embedding(text, model)

print("embeddings length: ", len(embedding))
print(embedding)

In [ ]:
model="text-embedding-ada-002"
df["embedding"] = df.comments.apply(lambda x: get_embedding(x, model=model))
df.to_csv("data/output/reviews-embeddings.csv")

In [ ]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_reviews(df, review_description, n=3, pprint=True):
   embedding = get_embedding(review_description, model='text-embedding-ada-002')
   df['similarities'] = df.embedding.apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values('similarities', ascending=False).head(n)
   return res

In [ ]:
results = search_reviews(df, "coffee")

results.head()


In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval

df_from_file = pd.read_csv('data/output/reviews-embeddings.csv')
df_from_file = df_from_file.dropna()
df_from_file["embedding"] = df_from_file.embedding.apply(literal_eval).apply(np.array)

results = search_reviews(df_from_file, "walkable")
results.head()